In [1]:
#숙제 2017, 10, 24 1. 이 문제는 교재의 3장 Exercise 중 9번 문제로 “auto.csv” (http://www-bcf.usc.edu/~gareth/ISL/data.html) 
#를 이용한 multiple linear regression에 관한 내용입니다. 
#(a) ~ (f) 문제 중 (d)번 문제를 제외하고 다 수행하십시오. 
#2. 이 문제는 교재의 4장 Exercise 중 13번 문제로 “Boston” 데이터를 이용한 Classification에 관한 것입니다. 
#내용은 어떤 지역의 범죄율 (crim) 이 그 중간값보다 높을까 또는 낮을까를 나타내는 예측모델을 만드는 것입니다. 
#적절한 변수들을 선택해 Logistic Regression과 KNN 모델을 만들어 데이터에 적용하고, 그 결과를 설명하십시오. 
#모델 설명 및 평가는 오늘 배울 내용인 train/test set 방법 및 K-fold cross validation을 사용해야 합니다.

#Using the Boston data set, fit classification models in order to predict whether a given suburb 
#has a crime rate above or below the median. 
#Explore logistic regression, LDA, and KNN models using various sub- sets of the predictors. 
#Describe your findings.

In [2]:
#homework 2, chapter 4 exercise13
# 패키지 imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import statsmodels.formula.api as smf
import scipy

%matplotlib inline

# import scikit-learn LogisticRegression estimator
from sklearn.linear_model import LogisticRegression as logisticregression

In [3]:
boston =pd.read_csv('../Boston.csv')
boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
crim       506 non-null float64
zn         506 non-null float64
indus      506 non-null float64
chas       506 non-null int64
nox        506 non-null float64
rm         506 non-null float64
age        506 non-null float64
dis        506 non-null float64
rad        506 non-null int64
tax        506 non-null int64
ptratio    506 non-null float64
black      506 non-null float64
lstat      506 non-null float64
medv       506 non-null float64
dtypes: float64(11), int64(3)
memory usage: 55.4 KB


In [4]:
#CRIM per capita crime rate by town
#ZN proportion of residential land zoned for lots over 25,000 sq.ft.
#INDUS proportion of non-retail business acres per town
#CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
#NOX nitric oxides concentration (parts per 10 million)
#RM average number of rooms per dwelling
#AGE proportion of owner-occupied units built prior to 1940
#DIS weighted distances to five Boston employment centres
#RAD index of accessibility to radial highways
#TAX full-value property-tax rate per $10,000
#PTRATIO pupil-teacher ratio by town
#B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#LSTAT % lower status of the population

#### 새로운 컬럼을 만들고 crim이 median보다 높으면 1, 낮으면 0

In [5]:
boston_crime_median = np.median(boston['crim'])

In [6]:
boston['crime_above_median'] = np.where(boston['crim'] > boston_crime_median,1,0)

In [7]:
boston.head(10)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv,crime_above_median
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,0
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,0
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,0
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,0
5,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7,0
6,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9,0
7,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15,27.1,0
8,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,16.5,0
9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,18.9,0


In [8]:
boston.nunique()

crim                  504
zn                     26
indus                  76
chas                    2
nox                    81
rm                    446
age                   356
dis                   412
rad                     9
tax                    66
ptratio                46
black                 357
lstat                 455
medv                  229
crime_above_median      2
dtype: int64

In [9]:
boston.describe(include='all')

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv,crime_above_median
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806,0.500000
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104,0.500495
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000,0.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000,0.000000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000,0.500000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000,1.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000,1.000000


In [10]:
boston.corr()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv,crime_above_median
crim,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305,0.409395
zn,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445,-0.436151
indus,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725,0.603260
chas,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260,0.070097
nox,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321,0.723235
rm,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,0.695360,-0.156372
age,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955,0.613940
dis,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929,-0.616342
rad,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626,0.619786
tax,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536,0.608741


In [11]:
boston.crime_above_median.sum()

253

In [12]:
#crim과 correlation이 높은 순서로 rad, tax, lstat, nox, indus, medv, black, dis, age, ptratio, rm, zn, chas

#### scikit learn의 logistic regression 모델

In [13]:
sk_logisticreg = logisticregression(solver='newton-cg')
x=boston.loc[:,['rad','tax','lstat','nox','indus','medv','black','dis','age','ptratio','rm','zn','chas']]
y=boston.crime_above_median
#fit the model
sk_logisticreg.fit(x,y)
print('classes: ', sk_logisticreg.classes_)
print('intercept :', sk_logisticreg.intercept_)
print('coefficient: ')
list(zip(x.columns, sk_logisticreg.coef_[0]) ) 

classes:  [0 1]
intercept : [-2.78211603]
coefficient: 


[('rad', 0.61532358727447234),
 ('tax', -0.0050077028967008934),
 ('lstat', 0.038476886094898695),
 ('nox', 2.0886725209661416),
 ('indus', 0.058048058268352659),
 ('medv', 0.056749019560806441),
 ('black', -0.013612013813105939),
 ('dis', -0.20811443093971826),
 ('age', 0.027438014073307839),
 ('ptratio', 0.055693598529802338),
 ('rm', 0.12228827979879345),
 ('zn', -0.053704157819134832),
 ('chas', -0.21792236976150245)]

In [14]:
#coefficient가 높은 순서대로 nox, rad, chas, dis ... 이므로 해당 4개의 variables만 사용하여 모델을 다시 fit
x_fit1=boston.loc[:,['nox','rad']]
y=boston.crime_above_median
#fit the model
sk_logisticreg.fit(x_fit1,y)
print('classes: ',sk_logisticreg.classes_)
print('intercept :', sk_logisticreg.intercept_)
print('coefficient: ')
list(zip(x_fit1.columns, sk_logisticreg.coef_[0]) ) 

classes:  [0 1]
intercept : [-5.3796336]
coefficient: 


[('nox', 5.7246595862231127), ('rad', 0.35956262161399533)]

In [15]:
#동일한 데이터 x에 대하여 모델을 돌려 보고 정확도 확인 
y_sk_test=sk_logisticreg.predict(x_fit1)

#crime rate가 median보다 높을것으로 추정한 숫자
print('crime rate가 median보다 높을것으로 추정한 숫자:',y_sk_test.sum())

#crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수
print('crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',(y==y_sk_test).sum())

#accuracy
print('accuracy:',(y==y_sk_test).mean())

crime rate가 median보다 높을것으로 추정한 숫자: 190
crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 421
accuracy: 0.832015810277


#### scikit learn logistic regression모델의 k-fold validation (5 fold로)

In [16]:
#K-fold cross validation
#약 500개의 observations를 5개의 fold로 나누어 validation

#테스트셋이 0~99

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[100:]
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
sk_logisticreg.fit(boston_train_x,boston_train_y)

#모델을 test
boston_test_fold=boston[:99]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=sk_logisticreg.predict(boston_test_x)
print('테스트셋 중 crime rate가 median보다 높을것으로 추정한 숫자:',boston_test_y.sum())
print('테스트셋 중 crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',(boston_test_fold.crime_above_median==boston_test_y).sum())
print('accuracy:',(boston_test_fold.crime_above_median==boston_test_y).mean())

테스트셋 중 crime rate가 median보다 높을것으로 추정한 숫자: 6
테스트셋 중 crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 71
accuracy: 0.717171717172


In [17]:
#테스트셋이 100~199

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:99].append(boston[200:])
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
sk_logisticreg.fit(boston_train_x,boston_train_y)

#모델을 test
boston_test_fold=boston[100:199]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=sk_logisticreg.predict(boston_test_x)
print('테스트셋 중 crime rate가 median보다 높을것으로 추정한 숫자:',boston_test_y.sum())
print('테스트셋 중 crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',(boston_test_fold.crime_above_median==boston_test_y).sum())
print('accuracy:',(boston_test_fold.crime_above_median==boston_test_y).mean())

테스트셋 중 crime rate가 median보다 높을것으로 추정한 숫자: 16
테스트셋 중 crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 69
accuracy: 0.69696969697


In [18]:
#테스트셋이 200~299

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:199].append(boston[300:])
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
sk_logisticreg.fit(boston_train_x,boston_train_y)

#모델을 test
boston_test_fold=boston[200:299]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=sk_logisticreg.predict(boston_test_x)
print('테스트셋 중 crime rate가 median보다 높을것으로 추정한 숫자:',boston_test_y.sum())
print('테스트셋 중 crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',(boston_test_fold.crime_above_median==boston_test_y).sum())
print('accuracy:',(boston_test_fold.crime_above_median==boston_test_y).mean())

테스트셋 중 crime rate가 median보다 높을것으로 추정한 숫자: 18
테스트셋 중 crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 81
accuracy: 0.818181818182


In [19]:
#테스트셋이 300~399

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:299].append(boston[400:])
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
sk_logisticreg.fit(boston_train_x,boston_train_y)

#모델을 test
boston_test_fold=boston[300:399]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=sk_logisticreg.predict(boston_test_x)
print('테스트셋 중 crime rate가 median보다 높을것으로 추정한 숫자:',boston_test_y.sum())
print('테스트셋 중 crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',(boston_test_fold.crime_above_median==boston_test_y).sum())
print('accuracy:',(boston_test_fold.crime_above_median==boston_test_y).mean())

테스트셋 중 crime rate가 median보다 높을것으로 추정한 숫자: 43
테스트셋 중 crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 85
accuracy: 0.858585858586


In [20]:
#테스트셋이 400~

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:399]
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
sk_logisticreg.fit(boston_train_x,boston_train_y)

#모델을 test
boston_test_fold=boston[400:]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=sk_logisticreg.predict(boston_test_x)
print('테스트셋 중 crime rate가 median보다 높을것으로 추정한 숫자:',boston_test_y.sum())
print('테스트셋 중 crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',(boston_test_fold.crime_above_median==boston_test_y).sum())
print('accuracy:',(boston_test_fold.crime_above_median==boston_test_y).mean())

테스트셋 중 crime rate가 median보다 높을것으로 추정한 숫자: 96
테스트셋 중 crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 101
accuracy: 0.952830188679


In [21]:
#k-fold validation 결과 0.72~0.90의 정확도를 보임

#### statsmodel의 logistic regression 모델

In [22]:
#statsmodel의 glm 사용
stats_logisticreg=smf.glm(formula='crime_above_median ~ zn + indus + chas + nox + rm +age + dis + rad + tax + ptratio + black + lstat + medv', data=boston).fit()
stats_logisticreg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:     crime_above_median   No. Observations:                  506
Model:                            GLM   Df Residuals:                      492
Model Family:                Gaussian   Df Model:                           13
Link Function:               identity   Scale:                   0.10118190769
Method:                          IRLS   Log-Likelihood:                -131.30
Date:                Mon, 13 Nov 2017   Deviance:                       49.781
Time:                        04:59:52   Pearson chi2:                     49.8
No. Iterations:                     2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.4619      0.357     -4.090      0.000      -2.162      -0.761
zn            -0.0013      0.001     -1.437      0.151      -0.003       0.000
indus          0.0030      0.004      0.732      0.464      -0.005       0.011
chas          -0.0154      0.058     -0.263      0.792      -0.130       0.099
nox            1.9908      0.261      7.639      0.000       1.480       2.502
rm             0.0219      0.030      0.723      0.470      -0.037       0.081
age            0.0028      0.001      3.120      0.002       0.001       0.004
dis            0.0100      0.014      0.717      0.473      -0.017       0.037
rad            0.0178      0.004      4.087      0.000       0.009       0.026
tax           -0.0002      0.000     -0.828      0.407      -0.001       0.000
ptratio        0.0123      0.009      1.332      0.183      -0.006       0.030
black         -0.0003      0.000     -1.381      0.167      -0.001       0.000
lstat          0.0037      0.004      0.980      0.327      -0.004       0.011
medv           0.0092      0.003      3.093      0.002       0.003       0.015
==============================================================================
"""

In [23]:
#P값이 낮은 nox, rad, age, medv 으로 다시 fit
stats_logisticreg=smf.glm(formula='crime_above_median ~ nox + rad', data=boston).fit()
stats_logisticreg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:     crime_above_median   No. Observations:                  506
Model:                            GLM   Df Residuals:                      503
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:                  0.107279266678
Method:                          IRLS   Log-Likelihood:                -151.70
Date:                Mon, 13 Nov 2017   Deviance:                       53.961
Time:                        04:59:52   Pearson chi2:                     54.0
No. Iterations:                     2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.9730      0.079    -12.341      0.000      -1.127      -0.818
nox            2.3748      0.159     14.940      0.000       2.063       2.686
rad            0.0163      0.002      7.706      0.000       0.012       0.020
==============================================================================
"""

In [24]:
stats_logisticreg_predict=stats_logisticreg.predict()
stats_logisticreg_predict_yesorno=[(1 if prob>0.5 else 0) for prob in stats_logisticreg_predict]
sum(stats_logisticreg_predict_yesorno)

#crime rate가 median보다 높을것으로 추정한 숫자
print('crime rate가 median보다 높을것으로 추정한 숫자:',sum(stats_logisticreg_predict_yesorno))

#crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수
print('crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',sum(y==stats_logisticreg_predict_yesorno))

#accuracy
print('accuracy:',sum(y==stats_logisticreg_predict_yesorno)/len(y))

crime rate가 median보다 높을것으로 추정한 숫자: 200
crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 431
accuracy: 0.851778656126


#### statsmodel logistic regression모델의 k-fold validation (5 fold로)

In [25]:
#K-fold cross validation
#약 500개의 observations를 5개의 fold로 나누어 validation

#테스트셋이 0~99
#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[100:]
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
stats_logisticreg=smf.glm(formula='crime_above_median ~ nox + rad', data=boston_train_fold).fit()

boston_test_fold=boston[:99]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=stats_logisticreg.predict(boston_test_x)
stats_logisticreg_predict_yesorno=[(1 if prob>0.5 else 0) for prob in boston_test_y]

print('crime rate가 median보다 높을것으로 추정한 숫자:',sum(stats_logisticreg_predict_yesorno))
print('crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',sum(boston_test_fold.crime_above_median==stats_logisticreg_predict_yesorno))
print('accuracy:',sum(boston_test_fold.crime_above_median==stats_logisticreg_predict_yesorno)/len(boston_test_fold.crime_above_median))

crime rate가 median보다 높을것으로 추정한 숫자: 0
crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 77
accuracy: 0.777777777778


In [26]:
#테스트셋이 100~199

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:99].append(boston[200:])
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
stats_logisticreg=smf.glm(formula='crime_above_median ~ nox + rad', data=boston_train_fold).fit()

#모델을 test
boston_test_fold=boston[100:199]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=stats_logisticreg.predict(boston_test_x)
stats_logisticreg_predict_yesorno=[(1 if prob>0.5 else 0) for prob in boston_test_y]

print('crime rate가 median보다 높을것으로 추정한 숫자:',sum(stats_logisticreg_predict_yesorno))
print('crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',sum(boston_test_fold.crime_above_median==stats_logisticreg_predict_yesorno))
print('accuracy:',sum(boston_test_fold.crime_above_median==stats_logisticreg_predict_yesorno)/len(boston_test_fold.crime_above_median))

crime rate가 median보다 높을것으로 추정한 숫자: 45
crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 96
accuracy: 0.969696969697


In [27]:
#테스트셋이 200~299

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:199].append(boston[300:])
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
stats_logisticreg=smf.glm(formula='crime_above_median ~ nox + rad', data=boston_train_fold).fit()

#모델을 test
boston_test_fold=boston[200:299]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=stats_logisticreg.predict(boston_test_x)
stats_logisticreg_predict_yesorno=[(1 if prob>0.5 else 0) for prob in boston_test_y]

print('crime rate가 median보다 높을것으로 추정한 숫자:',sum(stats_logisticreg_predict_yesorno))
print('crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',sum(boston_test_fold.crime_above_median==stats_logisticreg_predict_yesorno))
print('accuracy:',sum(boston_test_fold.crime_above_median==stats_logisticreg_predict_yesorno)/len(boston_test_fold.crime_above_median))

crime rate가 median보다 높을것으로 추정한 숫자: 10
crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 73
accuracy: 0.737373737374


In [28]:
#테스트셋이 300~399

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:299].append(boston[400:])
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
stats_logisticreg=smf.glm(formula='crime_above_median ~ nox + rad', data=boston_train_fold).fit()


#모델을 test
boston_test_fold=boston[300:399]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=stats_logisticreg.predict(boston_test_x)
stats_logisticreg_predict_yesorno=[(1 if prob>0.5 else 0) for prob in boston_test_y]

print('crime rate가 median보다 높을것으로 추정한 숫자:',sum(stats_logisticreg_predict_yesorno))
print('crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',sum(boston_test_fold.crime_above_median==stats_logisticreg_predict_yesorno))
print('accuracy:',sum(boston_test_fold.crime_above_median==stats_logisticreg_predict_yesorno)/len(boston_test_fold.crime_above_median))

crime rate가 median보다 높을것으로 추정한 숫자: 43
crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 85
accuracy: 0.858585858586


In [29]:
#테스트셋이 400~

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:399]
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
stats_logisticreg=smf.glm(formula='crime_above_median ~ nox + rad ', data=boston_train_fold).fit()


#모델을 test
boston_test_fold=boston[400:]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=stats_logisticreg.predict(boston_test_x)
stats_logisticreg_predict_yesorno=[(1 if prob>0.5 else 0) for prob in boston_test_y]

print('crime rate가 median보다 높을것으로 추정한 숫자:',sum(stats_logisticreg_predict_yesorno))
print('crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수:',sum(boston_test_fold.crime_above_median==stats_logisticreg_predict_yesorno))
print('accuracy:',sum(boston_test_fold.crime_above_median==stats_logisticreg_predict_yesorno)/len(boston_test_fold.crime_above_median))

crime rate가 median보다 높을것으로 추정한 숫자: 101
crime rate의 above-median, below-median 여부의 추정치와 실제가 일치하는 수: 96
accuracy: 0.905660377358


In [30]:
#k-fold validation 결과 0.73~0.89의 정확도를 보임

#### KNN을 이용한 classification model

In [31]:
#KNN and evaluation package import
#from sklearn.neighbors import KNeighborsRegressor as knnreg
from sklearn.neighbors import KNeighborsClassifier as knncls
from sklearn import metrics

In [32]:
x=boston.loc[:,['rad','tax','lstat','nox','indus','medv','black','dis','age','ptratio','rm','zn','chas']]
y=boston.crime_above_median

x_train=x[100:]
y_train=y[100:]
x_test=x[:100]
y_test=y[:100]

In [33]:
#KNN K=1
knn=knncls(n_neighbors=1)
knn.fit(x_train,y_train)
y_hat=knn.predict(x_test)
print ("test set error rate:",(y_test!=y_hat).mean())
conf_mat = metrics.confusion_matrix(y_test, y_hat)   # argument 순이 (True, Predicted)  
print(conf_mat)
TP = conf_mat[1,1]
TN = conf_mat[0,0]
FP = conf_mat[0,1]
FN = conf_mat[1,0]
print("Accuracy: ", (TP+TN) / (TP+TN+FP+FN) )
print("Sensitivity: ",  TP / (FN + TP) )
print("Precision: ",  TP / (FP + TP) )
print("False Positive Rate: ",  FP / (FP + TN) )

test set error rate: 0.19
[[59 19]
 [ 0 22]]
Accuracy:  0.81
Sensitivity:  1.0
Precision:  0.536585365854
False Positive Rate:  0.24358974359


In [34]:
#KNN K=5
knn=knncls(n_neighbors=5)
knn.fit(x_train,y_train)
y_hat=knn.predict(x_test)
print ("test set error rate:",(y_test!=y_hat).mean())
conf_mat = metrics.confusion_matrix(y_test, y_hat)   # argument 순이 (True, Predicted)  
print(conf_mat)
TP = conf_mat[1,1]
TN = conf_mat[0,0]
FP = conf_mat[0,1]
FN = conf_mat[1,0]
print("Accuracy: ", (TP+TN) / (TP+TN+FP+FN) )
print("Sensitivity: ",  TP / (FN + TP) )
print("Precision: ",  TP / (FP + TP) )
print("False Positive Rate: ",  FP / (FP + TN) )

test set error rate: 0.09
[[70  8]
 [ 1 21]]
Accuracy:  0.91
Sensitivity:  0.954545454545
Precision:  0.724137931034
False Positive Rate:  0.102564102564


In [35]:
#baseline error/accuracy 계산

def get_baseline_accuracy(y_train, y_test):   # returns (baseline accuracy, label) tuple  
    majority_label = y_train.value_counts().idxmax()
    baseline_acc = y_test.value_counts()[ majority_label ] / len(y_test)
    return((baseline_acc, majority_label))

print("Test set baseline error rate = %s,  with the label = \'%s\'" % 
      (1 - get_baseline_accuracy(y_train, y_test)[0], get_baseline_accuracy(y_train, y_test)[1]))

Test set baseline error rate = 0.78,  with the label = '1'


In [ ]:
k_range = range(1, 10)
precision_scores = []
for k in k_range:
    knn = knncls(n_neighbors=k)   
    knn.fit(x_train,y_train)
    y_predicted = knn.predict(x_test)
    conf_mat = metrics.confusion_matrix(y_test, y_predicted)
    TP = conf_mat[1,1]
    TN = conf_mat[0,0]
    FP = conf_mat[0,1]
    FN = conf_mat[1,0]
    precision_scores.append(TP/(FP + TP))
    
list(zip(k_range, precision_scores))


[(1, 0.53658536585365857),
 (2, 0.76923076923076927),
 (3, 0.66666666666666663),
 (4, 0.7142857142857143),
 (5, 0.72413793103448276),
 (6, 0.70370370370370372),
 (7, 0.7142857142857143),
 (8, 0.70370370370370372),
 (9, 0.72413793103448276)]

In [ ]:
x=boston.loc[:,['nox','rad','chas','dis']]
y=boston.crime_above_median

x_train=x[100:]
y_train=y[100:]
x_test=x[:100]
y_test=y[:100]

k_range = range(1, 10)
precision_scores = []
for k in k_range:
    knn = knncls(n_neighbors=k)   
    knn.fit(x_train,y_train)
    y_hat = knn.predict(x_test)
    conf_mat = metrics.confusion_matrix(y_test, y_hat)
    TP = conf_mat[1,1]
    TN = conf_mat[0,0]
    FP = conf_mat[0,1]
    FN = conf_mat[1,0]
    precision_scores.append(TP/(FP + TP))

list(zip(k_range, precision_scores))

In [ ]:
x=boston.loc[:,['nox','rad','age','medv']]
y=boston.crime_above_median

x_train=x[100:]
y_train=y[100:]
x_test=x[:100]
y_test=y[:100]

k_range = range(1, 10)
precision_scores = []
for k in k_range:
    knn = knncls(n_neighbors=k)   
    knn.fit(x,y)
    y_hat = knn.predict(x)
    conf_mat = metrics.confusion_matrix(y, y_hat)
    TP = conf_mat[1,1]
    TN = conf_mat[0,0]
    FP = conf_mat[0,1]
    FN = conf_mat[1,0]
    precision_scores.append(TP/(FP + TP))

list(zip(k_range, precision_scores))

In [ ]:
x=boston.loc[:,['nox','rad']]
y=boston.crime_above_median

x_train=x[100:]
y_train=y[100:]
x_test=x[:100]
y_test=y[:100]

k_range = range(1, 10)
precision_scores = []
for k in k_range:
    knn = knncls(n_neighbors=k)   
    knn.fit(x,y)
    y_hat = knn.predict(x)
    conf_mat = metrics.confusion_matrix(y, y_hat)
    TP = conf_mat[1,1]
    TN = conf_mat[0,0]
    FP = conf_mat[0,1]
    FN = conf_mat[1,0]
    precision_scores.append(TP/(FP + TP))

list(zip(k_range, precision_scores))

In [ ]:
#predictors를 nox, rad로 했을 때 가장 precision이 높음
#k값이 1,2,3,9일 때 가장 precision이 높음

#### KNN 모델의 k-fold validation
(k값을 3로 두고 predictors를 nox, rad로 했을 경우)

In [ ]:
#K-fold cross validation
#약 500개의 observations를 5개의 fold로 나누어 validation

#테스트셋이 0~99
#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[100:]
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
knn=knncls(n_neighbors=3).fit(boston_train_x,boston_train_y)

boston_test_fold=boston[:99]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=knn.predict(boston_test_x)

print ("accuracy:",(boston_test_fold.crime_above_median==boston_test_y).mean())

In [ ]:
#테스트셋이 100~199

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:99].append(boston[200:])
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
knn=knncls(n_neighbors=3).fit(boston_train_x,boston_train_y)

#모델을 test
boston_test_fold=boston[100:199]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=knn.predict(boston_test_x)

print ("accuracy:",(boston_test_fold.crime_above_median==boston_test_y).mean())

In [ ]:
#테스트셋이 200~299

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:199].append(boston[300:])
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
knn=knncls(n_neighbors=3).fit(boston_train_x,boston_train_y)

#모델을 test
boston_test_fold=boston[200:299]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=knn.predict(boston_test_x)

print ("accuracy:",(boston_test_fold.crime_above_median==boston_test_y).mean())

In [ ]:
#테스트셋이 300~399

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:299].append(boston[400:])
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
knn=knncls(n_neighbors=3).fit(boston_train_x,boston_train_y)


#모델을 test
boston_test_fold=boston[300:399]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=knn.predict(boston_test_x)

print ("accuracy:",(boston_test_fold.crime_above_median==boston_test_y).mean())

In [ ]:
#테스트셋이 400~

#400개의 observations로 모델을 fit한 후 나머지 100여개의 observations로 테스트
#트레인데이터로 모델 fit
boston_train_fold=boston[:399]
boston_train_x=boston_train_fold.loc[:,['nox','rad']]
boston_train_y=boston_train_fold.crime_above_median
knn=knncls(n_neighbors=3).fit(boston_train_x,boston_train_y)

#모델을 test
boston_test_fold=boston[400:]
boston_test_x=boston_test_fold.loc[:,['nox','rad']]
boston_test_y=knn.predict(boston_test_x)

print ("accuracy:",(boston_test_fold.crime_above_median==boston_test_y).mean())

In [ ]:
#k-fold validation 결과 0.75~0.93의 정확도를 보임